In [1]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
from sklearn.decomposition import PCA
from sklearn.externals import joblib
import sys, glob, math, os
import cv2
import numpy as np
import re

# image sizes
img_rows, img_cols = 224, 224

Using TensorFlow backend.


In [2]:
def vgg19_lower_model():
    # initial model, load vgg19
    vgg19 = VGG19(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
    vgg19.summary()

    # new model that takes first 10 layers of VGG19
    model = Sequential()
#    model.add(vgg19.get_layer('input_1'))
    model.add(vgg19.get_layer('block1_conv1'))
    model.add(vgg19.get_layer('block1_conv2'))
    model.add(vgg19.get_layer('block1_pool'))
    model.add(vgg19.get_layer('block2_conv1'))
    model.add(vgg19.get_layer('block2_conv2'))
    model.add(vgg19.get_layer('block2_pool'))
    model.add(vgg19.get_layer('block3_conv1'))
    model.add(vgg19.get_layer('block3_conv2'))
    model.add(vgg19.get_layer('block3_conv3'))
    model.add(vgg19.get_layer('block3_conv4'))
    model.add(vgg19.get_layer('block3_pool'))
    model.add(vgg19.get_layer('block4_conv1'))
    model.add(vgg19.get_layer('block4_conv2'))
    model.add(vgg19.get_layer('block4_conv3'))
    model.add(vgg19.get_layer('block4_conv4'))
#    model.add(vgg19.get_layer('block4_pool'))
#    model.add(vgg19.get_layer('block5_conv1'))
#    model.add(vgg19.get_layer('block5_conv2'))
#    model.add(vgg19.get_layer('block5_conv3'))
#    model.add(vgg19.get_layer('block5_conv4'))
#    model.add(vgg19.get_layer('block5_pool'))
#    model.add(Flatten())
#    model.summary()
 
    return model
#END vgg19_lower_model()

In [3]:
def load_imgs(dir,normalize=0):
    # load image data from dir

    data = []
    fname = []
    path  = dir + '/scene*.JPG'
    files = sorted(glob.glob(path))

    for fl in files:
        flbase = os.path.basename(fl)
        print('reading %s'%format(fl))

        img = cv2.imread(fl)
        img = cv2.resize(img, (img_rows,img_cols))
        img = np.array(img, dtype=np.float32)
        if normalize == 1:
            img -= np.mean(img)
            img /= np.std(img)

        data.append(img)
        fname.append(fl)

    data = np.array(data,dtype=np.float32)

    return data, fname
# end of load_imgs

In [5]:
if __name__=='__main__':

    # first create pca space from all data
    data,fname = load_imgs('../scenes', 1)

    # load eye and scene data
    data1,fname1 = load_imgs('dataset/scene',1) 
    print(data.shape)
    data2,fname2 = load_imgs('dataset/eye',1)
    print(data2.shape)    

reading ../scenes/scene007a.JPG
reading ../scenes/scene007b.JPG
reading ../scenes/scene007c.JPG
reading ../scenes/scene007d.JPG
reading ../scenes/scene007e.JPG
reading ../scenes/scene007f.JPG
reading ../scenes/scene007g.JPG
reading ../scenes/scene008a.JPG
reading ../scenes/scene008b.JPG
reading ../scenes/scene009a.JPG
reading ../scenes/scene009b.JPG
reading ../scenes/scene009c.JPG
reading ../scenes/scene009d.JPG
reading ../scenes/scene009e.JPG
reading ../scenes/scene009f.JPG
reading ../scenes/scene010a.JPG
reading ../scenes/scene010b.JPG
reading ../scenes/scene010c.JPG
reading ../scenes/scene010d.JPG
reading ../scenes/scene011a.JPG
reading ../scenes/scene011b.JPG
reading ../scenes/scene011c.JPG
reading ../scenes/scene012a.JPG
reading ../scenes/scene012b.JPG
reading ../scenes/scene012c.JPG
reading ../scenes/scene012d.JPG
reading ../scenes/scene013a.JPG
reading ../scenes/scene013b.JPG
reading ../scenes/scene013c.JPG
reading ../scenes/scene013d.JPG
reading ../scenes/scene014a.JPG
reading 

In [6]:
    model = vgg19_lower_model()
    
    # all data
    out = model.predict(data, batch_size=32, verbose=1)
    print(out.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=50)
    out = np.reshape(out,(out.shape[0]*out.shape[1]*out.shape[2],out.shape[3]))    
    out -= np.mean(out)
    out /= np.std(out)
    pca.fit(out)
    joblib.dump(pca, 'pca.pkl') 

['pca.pkl']

In [8]:
    # eye and scene data
    out1 = model.predict(data1, batch_size=32, verbose=1)
    out1 = np.reshape(out1,(out1.shape[0]*out1.shape[1]*out1.shape[2],out1.shape[3]))
    out1 -= np.mean(out1)
    out1 /= np.std(out1)
    out2 = model.predict(data2, batch_size=32, verbose=1)
    out2 = np.reshape(out2,(out2.shape[0]*out2.shape[1]*out2.shape[2],out2.shape[3]))
    out2 -= np.mean(out2)
    out2 /= np.std(out2)    
    print(out1.shape)
    print(out2.shape)

6/6 [==============================] - 0s
(4704, 512)
(4704, 512)


In [9]:
X_pca = pca.transform(out1)
Y_pca = pca.transform(out2)

np.save("scene.npy",out1)
np.save("eye.npy",out2)
np.save("X_pca.npy", X_pca)
np.save("Y_pca.npy", Y_pca)